## LINEAR REGRESSION - HỒI QUY TUYẾN TÍNH

> **Sử dụng dataset: ds_salaries.csv**

Dự đoán các yếu tố để nhân viên được trả lương cao hơn

Import thư viện

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

Đọc dữ liệu từ file csv

In [2]:
df = pd.read_csv('ds_salaries.csv')

### 1. Thực hiện hồi quy tuyến tính từ Scikit-learn (dùng hàm LinearRegression()) cho trường hợp 1 biến và 2 biến.

**Chuẩn bị bộ dữ liệu cho mô hình hồi quy tuyến tính:**

- Loại bỏ ngoại lệ ra khỏi bộ dữ liệu sử dụng phương pháp IQR

- Một trong những giả định chính của hồi quy tuyến tính là biến kết quả có phân phối chuẩn. Nếu biến kết quả không có phân phối chuẩn, nó có thể dẫn đến ước tính sai lệch hoặc không hiệu quả về các hệ số hồi quy và sai số chuẩn tăng cao. Để biến các giá trị dị thường trở nên bình thường, hãy chuyển đổi biến kết quả (ví dụ: biến đổi nhật ký, Box-Cox).

- Chia tỷ lệ biến kết quả.

- Tách bộ dữ liệu gốc để tạo bộ dữ liệu training, validation và test.

In [8]:
# Tạo 1 bản sao cho dataset gốc
df_copy = df.copy()

# Xóa các cột không cần thiết để chuẩn bị cho việc train model
df_copy = df_copy.drop(['work_year','employee_residence','remote_ratio'], axis=1)

# In ra số dòng và số cột của dataset lúc sau
df_copy.shape 

(3755, 8)

**Loại bỏ ngoại lệ**

In [9]:
Q1 = df_copy['salary_in_usd'].quantile(0.25)
Q3 = df_copy['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1
df_copy = df_copy[~((df_copy['salary_in_usd']< (Q1 - 1.5 * IQR)) | (df_copy['salary_in_usd'] > (Q3 + 1.5 * IQR)))]
df_copy.shape

(3692, 8)

**Tách dữ liệu**

Tách bộ dữ liệu thành 2 phần: 1 phần để train model (80%) và 1 phần để test model (20%)

In [13]:
categorical_vars = ['experience_level','job_title' ,'company_location','company_size','employment_type']
numeric_vars = ['salary_in_usd']

In [14]:
X = df_copy[categorical_vars]
#X = df_copy[categorical_vars].copy().reset_index(drop=True)

X.head()

,experience_level,job_title,company_location,company_size,employment_type
0,SE,Principal Data Scientist,ES,L,FT
1,MI,ML Engineer,US,S,CT
2,MI,ML Engineer,US,S,CT
3,SE,Data Scientist,CA,M,FT
4,SE,Data Scientist,CA,M,FT


In [15]:
y = df_copy[numeric_vars]
y.head()

,salary_in_usd
0,85847
1,30000
2,25500
3,175000
4,120000


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Trước khi đưa dữ liệu vào mô hình hồi quy tuyến tính, cần phải chuẩn hóa dữ liệu. Các giá trị ở cột biến định tính cần được mã hóa thành nhị phân. Điều này được thực hiện bằng cách tiền xử lý dữ liệu và phương pháp One Hot Encode

**Tiền xử lý dữ liệu**

Chuyển đổi giá trị ở biến định lượng

In [28]:
df_copy['salary_in_usd'] = np.log1p(df_copy['salary_in_usd'])

e:\LAP TRINH PYTHON\LT PTDL 1\venv\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_copy['salary_in_usd'] = scaler.fit_transform(df_copy[['salary_in_usd']]) 

Chuyển đổi các giá trị ở biến định lượng

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

cat_cols = df_copy.select_dtypes(include=['object']).columns
num_cols = df_copy.select_dtypes(include=['float64']).columns

categorical_transformer = Pipeline(steps=[("hotencoder", OneHotEncoder(handle_unknown="ignore"))])

In [32]:
df_copy.salary_in_usd.dtype
cat_cols
num_cols

Index(['salary_in_usd'], dtype='object')

Gom nhóm các giá trị ở biến định lượng và định tính vào pipeline

In [33]:
#preprocessor pipeline

from sklearn.compose import ColumnTransformer

#preprocessor = ColumnTransformer(transformers=[
     #   ('cat', categorical_transformer, cat_cols),('num', 'passthrough', num_cols)])

preprocessor = ColumnTransformer(transformers=[ ("cat", categorical_transformer, cat_cols) ])

**Xây dựng mô hình hồi quy tuyến tính**

In [34]:
# Tạo mô hình hồi quy tuyến tính

from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [39]:
# Tạo pipeline

from sklearn.pipeline import Pipeline

lr_train = Pipeline(steps = [("preprocessor", preprocessor), ("lr", lr)])

# Làm khớp mô hình với dữ liệu
reg_model = lr_train.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe

ValueError: A given column is not a column of the dataframe

#### **Hồi quy tuyến tính 1 biến sử dụng hàm LinearRegression()**

**Hồi quy tuyến tính 1 biến:**
Hãy cho biết sự ảnh hưởng của tổng lượng công việc được làm từ xa đến mức lương của nhân viên.

1. Xác định biến phụ thuộc và biến độc lập
- Biến phụ thuộc: mức lương
- Biến độc lập: vị trí công việc

2. Y = f(X) + A0 + A1*X + epsilon

3. Chạy mô hình
4. Đọc và hiểu kết quả

In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np

model = LinearRegression()

X = df[['remote_ratio']]
Y = df['salary_in_usd']

model.fit(X, Y)


print('Intercept:', model.intercept_)
print('Coefficient:', model.coef_[0])
print('R-squared:', model.score(X, Y))


# print(model.intercept_, model.coef_, model.score(X, Y))

Intercept: 141423.7447202622
Coefficient: -83.2768197098398
R-squared: 0.004117915340328904


**Nhận xét:**

- Ta có phương trình cho mô hình hồi quy tuyến tính 1 biến là:

    Y = Intercept + Coefficient * X

    Y = 141423.7447 -83.2768 * X

Từ đó suy ra: R^2 = 0.0041179. Giá trị R-squared càng gần 1 thì mô hình càng giải thích tốt mối quan hệ giữa biến độc lập và biến phụ thuộc.

- Mối tương quan giữa biến X (trong trường hợp này là G2) và biến Y (trong trường hợp này là G3) có thể được đánh giá dựa trên R-squared. Nếu R-squared càng gần 1, đồng nghĩa với việc mối tương quan giữa biến X và biến Y càng mạnh.

2. Thực hiện hồi quy tuyến tính bằng phương pháp StaSmodels (Dùng hàm OLS()) cho trường hợp 1 biến và 2 biến.

#### **Hồi quy tuyến tính 2 biến sử dụng hàm LinearRegression()**

**Hồi quy tuyến tính 2 biến:**
Hãy cho biết sự ảnh hưởng của điểm môn đọc và điểm môn viết đến điểm môn toán của sinh viên

1. Xác định biến phụ thuộc và biến độc lập
- Biến phụ thuộc: điểm môn toán
- Biến độc lập: điểm môn viết điểm môn đọc

2. Y = f(X) + A0 + A1*X + epsilon

3. Chạy mô hình
4. Đọc và hiểu kết quả